In [ ]:
import json
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import string

# Load the dictionary file
dictionary_file_path = 'kamus-2cba6-export.json'

with open(dictionary_file_path, 'r') as file:
    data = json.load(file)
    dictionary_data = data['employees']  

def count_words(sentence, dictionary):
    words = sentence.split()
    ngoko_count = 0
    kramaalus_count = 0
    kramainggil_count = 0
    
    for word in words:
        word = word.strip(string.punctuation)
        for entry in dictionary.values():
            ngoko_word = entry.get('ngoko', '').lower()
            kramaalus_word = entry.get('kramaalus', '').lower()
            kramainggil_word = entry.get('kramainggil', '').lower()
            
            if word.lower() == ngoko_word:
                ngoko_count += 1
                break
            elif word.lower() == kramaalus_word:
                kramaalus_count += 1
                break
            elif word.lower() == kramainggil_word:
                kramainggil_count += 1
                break
    
    return ngoko_count, kramaalus_count, kramainggil_count

def classify_sentence(sentence, dictionary):
    ngoko_count, kramaalus_count, kramainggil_count = count_words(sentence, dictionary)
    
    total_classified_words = ngoko_count + kramaalus_count + kramainggil_count
    if total_classified_words == 0:
        return -1  # "Unknown" for sentences that don't fit any category

    ngoko_proportion = ngoko_count / total_classified_words
    kramaalus_proportion = kramaalus_count / total_classified_words
    kramainggil_proportion = kramainggil_count / total_classified_words
    
    # Determine initial classification based on proportions
    if ngoko_proportion > kramaalus_proportion and ngoko_proportion > kramainggil_proportion:
        classification = "ngoko lugu"
    elif kramaalus_proportion >= ngoko_proportion and kramaalus_proportion > kramainggil_proportion:
        classification = "krama lugu"
    else:
        classification = "krama alus"
    
    # Apply tie-breaking rules
    if ngoko_proportion == kramaalus_proportion:
        classification = "ngoko lugu"
    
    # Upgrade classification if kramainggil words are present
    if kramainggil_count > 0:
        if classification == "ngoko lugu":
            classification = "ngoko alus"
        elif classification == "krama lugu":
            classification = "krama alus"
    
    # Downgrade classification if ngoko words are present
    if ngoko_count > 0:
        if classification == "krama alus":
            classification = "krama lugu"
        elif classification == "krama lugu":
            classification = "ngoko alus"
    
    # Map classification to numerical labels
    classification_map = {
        "ngoko lugu": 0,
        "ngoko alus": 1,
        "krama lugu": 2,
        "krama alus": 3
    }
    
    return classification_map.get(classification, -1)

def evaluate_performance(test_file_path, dictionary):
    # Load test data
    df_test = pd.read_csv(test_file_path)
    
    # Classify sentences and compare with labels
    df_test['predicted'] = df_test['sentence'].apply(lambda x: classify_sentence(x, dictionary))
    
    # Get unclassified sentences
    unclassified = df_test[df_test['predicted'] == -1]
    unclassified_count = len(unclassified)
    unclassified_examples = unclassified['sentence'].tolist()
    
    # Filter out unclassified results
    df_test = df_test[df_test['predicted'] != -1]
    
    # Calculate metrics
    accuracy = accuracy_score(df_test['label'], df_test['predicted'])
    precision = precision_score(df_test['label'], df_test['predicted'], average='weighted')
    recall = recall_score(df_test['label'], df_test['predicted'], average='weighted')
    conf_matrix = confusion_matrix(df_test['label'], df_test['predicted'])
    
    return accuracy, precision, recall, conf_matrix, unclassified_count, unclassified_examples

def main():
    print("Evaluating Dictionary-based Classifier")
    
    # Path to the test file
    test_file_path = 'df_test_group.csv'
    
    # Evaluate performance
    accuracy, precision, recall, conf_matrix, unclassified_count, unclassified_examples = evaluate_performance(test_file_path, dictionary_data)
    
    # Print results
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print(f"Number of Unclassified Sentences: {unclassified_count}")
    print("Examples of Unclassified Sentences:")
    for example in unclassified_examples[:5]:  # Show up to 5 examples
        print(f"- {example}")

if __name__ == "__main__":
    main()